In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
ufo = pd.read_json('../data/ufo_first100records.json', lines=True)

In [12]:
ufo

,_id,url,html,time
0,{'$oid': '591382287c40e83fd74240bd'},http://www.nuforc.org/webreports/133/S133978.html,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\...",Wed May 10 21:12:07 2017
1,{'$oid': '591389377c40e840c123dfef'},http://www.nuforc.org/webreports/133/S133977.h...,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\...",Wed May 10 21:42:15 2017
2,{'$oid': '591389397c40e840c123dff0'},http://www.nuforc.org/webreports/133/S133974.h...,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\...",Wed May 10 21:42:17 2017
3,{'$oid': '5913893c7c40e840c123dff1'},http://www.nuforc.org/webreports/133/S133964.h...,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\...",Wed May 10 21:42:20 2017
4,{'$oid': '5913893f7c40e840c123dff2'},http://www.nuforc.org/webreports/133/S133962.h...,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\...",Wed May 10 21:42:23 2017
...,...,...,...,...
94,{'$oid': '59138a2f7c40e840c123e04c'},http://www.nuforc.org/webreports/133/S133980.h...,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\...",Wed May 10 21:46:23 2017
95,{'$oid': '59138a317c40e840c123e04d'},http://www.nuforc.org/webreports/133/S133923.h...,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\...",Wed May 10 21:46:25 2017
96,{'$oid': '59138a347c40e840c123e04e'},http://www.nuforc.org/webreports/133/S133928.h...,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\...",Wed May 10 21:46:28 2017
97,{'$oid': '59138a377c40e840c123e04f'},http://www.nuforc.org/webreports/133/S133891.h...,"<html xmlns:v=""urn:schemas-microsoft-com:vml""\...",Wed May 10 21:46:31 2017


In [13]:
ufo_html = ufo.html
ufo_html[0]

'<html xmlns:v="urn:schemas-microsoft-com:vml"\r\nxmlns:o="urn:schemas-microsoft-com:office:office"\r\nxmlns:w="urn:schemas-microsoft-com:office:word"\r\nxmlns:m="http://schemas.microsoft.com/office/2004/12/omml"\r\nxmlns="http://www.w3.org/TR/REC-html40">\r\n\r\n<head>\r\n<meta http-equiv=Content-Type content="text/html; charset=windows-1252">\r\n<meta name=ProgId content=Word.Document>\r\n<meta name=Generator content="Microsoft Word 15">\r\n<meta name=Originator content="Microsoft Word 15">\r\n<link rel=File-List href="STemplate_files/filelist.xml">\r\n<!--[if gte mso 9]><xml>\r\n <o:DocumentProperties>\r\n  <o:Author>Christian Stepien</o:Author>\r\n  <o:LastAuthor>Christian Stepien</o:LastAuthor>\r\n  <o:Revision>3</o:Revision>\r\n  <o:TotalTime>1</o:TotalTime>\r\n  <o:Created>2014-02-14T21:30:00Z</o:Created>\r\n  <o:LastSaved>2014-02-14T21:35:00Z</o:LastSaved>\r\n  <o:Pages>1</o:Pages>\r\n  <o:Words>58</o:Words>\r\n  <o:Characters>331</o:Characters>\r\n  <o:Lines>2</o:Lines>\r\n  <

In [15]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(ufo_html[0], 'html.parser')

In [16]:
table=soup.find('tbody')

In [31]:
X = table.text.strip()


In [35]:
import re
def data_pipeline(data):
    '''
    input: raw dataframe with html column

    '''

    contents = []
    for row in data.html:
        soup = BeautifulSoup(row, "html.parser")
        table = soup.find("tbody")
        text = table.text.strip()
        a = re.findall(r'(Duration.+?\n\n\n)(.+)(\(\(NUFORC Note:)?', text)
        try:
            contents.append(a[0][1])
        except:
            pass

    return contents

contents = data_pipeline(ufo)

In [37]:
data = pd.DataFrame(contents)

In [41]:
X = data[0]
X

0     Light seen over mountain's east of Camp McGreg...
1     Light in sky stationary.  Not a airplane or an...
2     Flying saucer descends, possibly lands in Nort...
3     While letting my dog out, a very bright white ...
4     A fire ball was moving in the atmosphere while...
                            ...                        
94    We got home from working the 2-10 shift at sec...
95    Brown saucer with red and green lights seen in...
96    There was a reflection on the window of a circ...
97    i was observing what I thought to be a large e...
98    TR3B Absolutely Certain!I saw the TR3B in the ...
Name: 0, Length: 99, dtype: object

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF

# removes punctuation and stopwords. replaces strings with lowercase and outputs top 1000 features. 
tfidf = TfidfVectorizer(lowercase=True, stop_words='english', max_features = 1000)

tfidf_ngrams = TfidfVectorizer(lowercase=True, stop_words='english', max_features = 1000, ngram_range=(1,2))

count_vec = TfidfVectorizer(lowercase=True, stop_words='english', max_features = 1000)

count_vec_ngrams = TfidfVectorizer(lowercase=True, stop_words='english', max_features = 1000, ngram_range=(1,2))

for transformers in [tfidf,tfidf_ngrams,count_vec,count_vec_ngrams]:
    transformers.fit(X)
    

ImportError: cannot import name 'NMF' from 'sklearn.cluster' (/home/daniel/anaconda3/lib/python3.7/site-packages/sklearn/cluster/__init__.py)